In [ ]:
!git clone https://github.com/theNocturni/NLP_musicBERT_rec

Cloning into 'PiRhDy'...
remote: Enumerating objects: 1657, done.
remote: Counting objects: 100% (1539/1539), done.
remote: Compressing objects: 100% (1461/1461), done.
remote: Total 1657 (delta 96), reused 1477 (delta 64), pack-reused 118
Receiving objects: 100% (1657/1657), 265.21 MiB | 16.05 MiB/s, done.
Resolving deltas: 100% (133/133), done.
Checking out files: 100% (1524/1524), done.


In [ ]:
!git clone https://github.com/pytorch/fairseq

Cloning into 'fairseq'...
remote: Enumerating objects: 33460, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 33460 (delta 14), reused 35 (delta 6), pack-reused 33407
Receiving objects: 100% (33460/33460), 23.33 MiB | 14.33 MiB/s, done.
Resolving deltas: 100% (24407/24407), done.


In [ ]:
!cd fairseq;git checkout 336942734c85791a90baa373c212d27e7c722662;pip install --editable .

Note: checking out '336942734c85791a90baa373c212d27e7c722662'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 33694273 Better support for WandB (#1530)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 123 kB 31.8 MB/s 
     |████████████████████████████████| 118 kB 71.6 MB/s 
     |████████████████████████████████| 112 kB 77.3 MB/s 


In [ ]:
! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"
# os.environ['PYTHONPATH'] += ":/content/fairseq/fairseq"

! echo $PYTHONPATH

/env/python
/env/python:/content/fairseq/


### Copy checkpoint data from google drive(can be skipped if manually uploading data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd NLP_musicBERT_rec/musicbert

/content/PiRhDy/musicbert


In [ ]:
!mkdir checkpoints

In [ ]:
# !cp /content/drive/MyDrive/checkpoint_last_musicbert_base.pt ./checkpoints
!cp /content/drive/MyDrive/checkpoint10_nsp_acc_checkpoint_last_musicbert_base.pt ./checkpoints
!cp -r /content/drive/MyDrive/acc_augdata_bin .

## Fine Tuning

In [ ]:
!bash train_nsp.sh acc checkpoints/checkpoint_last_musicbert_base.pt

task = acc
model = checkpoints/checkpoint_last_musicbert_base.pt
disable_cp = False
mask_strategy = ['bar']
convert_encoding = OCTMIDI
crop_length = None
2022-12-05 16:41:11 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'simple', 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': 'musicbert', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': True}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': N

## Copy the checkpoint file to Google Drive

In [ ]:
!cp ./checkpoints/checkpoint_last_nsp_acc_checkpoint_last_musicbert_base.pt /content/drive/MyDrive/ 

## Evaluation

In [ ]:
# only inference, no saving logit
# !python -u eval_nsp.py checkpoints/checkpoint10_nsp_acc_checkpoint_last_musicbert_base.pt acc_data_bin

loading model and data
disable_cp = False
mask_strategy = ['bar']
convert_encoding = OCTMIDI
crop_length = None
1042301B [00:01, 903211.55B/s]
456318B [00:00, 514218.98B/s]
evaluating: 99.9804%
label[0] = 0
label[1] = 1
y_true.shape = (30550, 2)
y_pred.shape = (30550, 2)
MAP: 0.884739
HITS@1: 0.181578
HITS@5: 0.731048
HITS@10: 0.922597
HITS@15: 0.953630
HITS@20: 0.967334
HITS@25: 0.980889


In [ ]:
#saves logits as logit.txt
!python -u test.py checkpoints/checkpoint10_nsp_acc_checkpoint_last_musicbert_base.pt acc_augdata_bin

loading model and data
disable_cp = False
mask_strategy = ['bar']
convert_encoding = OCTMIDI
crop_length = None
1042301B [00:01, 941787.62B/s]
456318B [00:00, 531231.25B/s]

label[0] = 0
label[1] = 1
y_true.shape = (2316950, 2)
y_pred.shape = (2316950, 2)
MAP: 0.797405
HITS@1: 0.162514
HITS@5: 0.607809
HITS@10: 0.840498
HITS@15: 0.917161
HITS@20: 0.950786
HITS@25: 0.968622


## Analysis

In [ ]:
%cd ../

s:\NLP\PiRhDy


In [ ]:
import copy
import miditoolkit


In [ ]:
import pickle
import numpy as np
import random
import os

def check_phrase(matrix, threshold_value):
    count_num = len(np.nonzero(matrix)[0]) // 4
    if count_num / matrix.shape[1] >= threshold_value:
        return True
    return False

dirs = ['0','1','2','3','4','5','6','7','8','9']
total = 0
acc_num_list_total = []
test_melody_harmony = []

for name in dirs:
    seq_list = pickle.load(open('filenames/sequence/{}'.format(name), 'rb'))
    file_list = pickle.load(open('filenames/phrase/{}'.format(name), 'rb'))
    for file_id, file in enumerate(file_list):
        acc_num_list = []
        sequence = np.load(seq_list[file_id], allow_pickle=True)
        phrases = np.load(file, allow_pickle=True)
        meta_dict = json.loads(sequence['meta.json'].decode('utf-8'))
        for idx, k in enumerate(meta_dict.keys()):
            if meta_dict[k] == 0:
                track_index_0 = idx
            elif meta_dict[k] == 25:
                track_index_1 = idx
            elif meta_dict[k] == 33:
                track_index_2 = idx
        phrase_id = 0
        if file_id % 100 not in [0,1,2,3,4,5]:
            while str(phrase_id) in phrases:
                acc_num = 0
                phrase = phrases[str(phrase_id)]
                melody = phrase[0:4, :]
                inst_0 = phrase[4*track_index_0+4:4*track_index_0+8,3]
                inst_1 = phrase[4*track_index_1+4:4*track_index_1+8,3]
                inst_2 = phrase[4*track_index_2+4:4*track_index_2+8,3]

                if sum(inst_0) * sum(inst_1) * sum(inst_2) == 0:
                    include_list.append(0)
                else:
                    include_list.append(1)
                track_num = phrase.shape[0] // 4
                for track_id in range(1, track_num):
                    accompaniment = phrase[0 + track_id * 4:4 * (track_id + 1), :]
                    if check_phrase(accompaniment, 0.5):
                        test_melody_harmony.append([file_id, melody, accompaniment])
                        acc_num += 1
                if acc_num != 0:
                    acc_num_list.append(acc_num)
                phrase_id += 1
            acc_num_list_total.append(acc_num_list)
print(len(acc_num_list_total))
# for l in acc_num_list_total:
#     print(len(l))
print(len(test_melody_harmony))

852
84652


In [ ]:
print(len(acc_num_list_total)/6)

142.0


s:\NLP\PiRhDy


In [ ]:
%cd musicbert

s:\NLP\PiRhDy\musicbert


In [ ]:
# %cd musicbert
import numpy as np
from numpy import dot
from numpy.linalg import norm

candidate = ['add_neg','add_pos','change_neg','change_pos','change_shuf_neg','orig']
result = np.loadtxt('logits.txt')
# print(result.shape)
start = 0
for i in range(10):
# for i in range(int(len(acc_num_list_total)/5)):
    logits = []
    for j in range(6):
        logits.append(start+result[len(acc_num_list_total[6*i]) * 50 * j:len(acc_num_list_total[6*i]) * 50 * (j+1)])
    start = len(acc_num_list_total[6*i]) * 50 * 6
    print(f'#{6*i+1}')
    for j in range(6):
        cos_sim = []
        avg_logit = []
        for k in range(len(logits[j])):
            avg_logit.append(logits[j][k])
            # if j in [0,2]:
            #     cos_sim.append(dot(logits[j][k], logits[j+1][k])/(norm(logits[j][k])*norm(logits[j+1][k])))    
            # elif j in [1,3,4]:
            #     cos_sim.append(dot(logits[j][k], logits[j-1][k])/(norm(logits[j][k])*norm(logits[j-1][k])))    
            # else:
            #     cos_sim.append(dot(logits[j][k], logits[5][k])/(norm(logits[j][k])*norm(logits[5][k])))
            cos_sim.append(dot(logits[j][k], logits[5][k])/(norm(logits[j][k])*norm(logits[5][k])))
        # print(sum(avg_logit)/len(avg_logit))
        print('candidate: ',candidate[j])
        print('max: ',max(cos_sim))
        print('min: ',min(cos_sim))
        print('mean: ',sum(cos_sim)/len(cos_sim))
        print('')


#1
candidate:  add_neg
max:  0.9999998690625723
min:  -0.9999999179083262
mean:  0.18175010669342107

candidate:  add_pos
max:  0.9999999979673172
min:  -0.9999999130765607
mean:  0.23433659781466923

candidate:  change_neg
max:  0.999999998448033
min:  -0.9999998769328665
mean:  0.284694093815166

candidate:  change_pos
max:  0.999999988153312
min:  -0.9999999786252992
mean:  0.28692547561721377

candidate:  change_shuf_neg
max:  0.9999999973915049
min:  -0.9999999292227921
mean:  0.22438809217822867

candidate:  orig
max:  1.0000000000000002
min:  0.9999999999999998
mean:  1.0

#7
candidate:  add_neg
max:  0.9999999999993612
min:  0.9999990988507839
mean:  0.9999998719965212

candidate:  add_pos
max:  0.9999999999986374
min:  0.999998976206238
mean:  0.9999998845133299

candidate:  change_neg
max:  1.0000000000000002
min:  0.9999991714108962
mean:  0.9999998904184115

candidate:  change_pos
max:  0.9999999999912269
min:  0.9999992742666743
mean:  0.9999998813507952

candidate:  chang